### Create body part category dictionary

This script will create a bodypart category dictionary and allow for addition of new keywords as needed.
Let's start by importing the libraries required.

In [22]:
import pandas as pd
import glob
import os
import csv
import numpy as np

from ipywidgets import *
from tkinter import Tk, filedialog
from IPython.display import clear_output, display

Let's create a dictionary with fixed categories as per Daniel Serra's spreadsheet.

In [23]:
# Create a template for the dictionary

BodyPart_Category = {
    'Abdomen': [],
    'Ankle': [],
    'Chest': [],
    'Clavicle': [],
    'C-Spine': [],
    'Elbow': [],
    'Foot': [],
    'Forearm':[],
    'Hand':[],
    'Hip':[],
    'Knee':[],
    'L-Spine':[],
    'Neck':[],
    'Pelvis':[],
    'Shoulder':[],
    'Skull':[],
    'Face':[],
    'Sacrum/Coccyx':[],
    'Tib/Fib':[],
    'T-Spine':[],
    'Wrist':[],
    'Femur':[],
    'Humerus':[],
    'QC/Test':[],
}

# Print the dictionary to see its initial state
print(BodyPart_Category)


{'Abdomen': [], 'Ankle': [], 'Chest': [], 'Clavicle': [], 'C-Spine': [], 'Elbow': [], 'Foot': [], 'Forearm': [], 'Hand': [], 'Hip': [], 'Knee': [], 'L-Spine': [], 'Neck': [], 'Pelvis': [], 'Shoulder': [], 'Skull': [], 'Face': [], 'Sacrum/Coccyx': [], 'Tib/Fib': [], 'T-Spine': [], 'Wrist': [], 'Femur': [], 'Humerus': [], 'QC/Test': []}


Input the merged database containing the collated reject logs.

In [24]:
# Select input file 

def select_files(b):
    clear_output()
    root = Tk()
    root.withdraw() # Hide the main window.
    root.call('wm', 'attributes', '.', '-topmost', True) # Raise the root to the top of all windows.
    b.files = filedialog.askopenfilename() # List of selected files will be set button's file attribute.
    print(b.files) # Print the list of files selected.

fileselect = Button(description="File select")
fileselect.on_click(select_files)

display(fileselect)

C:/Users/bernardm/GitHub/JupyterNotebooks/rejectAnalysis/outputdata/merged_output.csv


In [30]:
files = fileselect.files

df = pd.read_csv(files)
df

,Asset Number,DeviceID,Manufacturer,Model,Image Date,Image Time,Body Part,View,Exposure Index,KAP (uGy.m2),kVp,Exposure (mAs),Exposure time (ms),Image Status,Reject Reason
0,12345667,QHSCHDXC07,Philips,Digital Diagnost,02/07/2020,10:39:58 AM,Wrist L,Lateral,473,3.23,57.0,3.0,10.0,confirmed,NaN
1,12345667,QHSCHDXC07,Philips,Digital Diagnost,02/07/2020,10:39:25 AM,Wrist L,PA,243,1.5,55.0,2.0,10.0,confirmed,NaN
2,12345667,QHSCHDXC07,Philips,Digital Diagnost,02/07/2020,11:19:54 AM,Wrist L,Lateral,615,3.05,57.0,3.0,10.0,rejected,Not True Lateral
3,12345667,QHSCHDXC07,Philips,Digital Diagnost,02/07/2020,11:19:18 AM,Wrist L,Lateral,565,3.05,57.0,3.0,10.0,confirmed,NaN
4,12345667,QHSCHDXC07,Philips,Digital Diagnost,02/07/2020,11:18:54 AM,Wrist L,PA,416,1.87,55.0,2.0,10.0,confirmed,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21665,123,NaN,Siemens,Ysio X.Pree,2022-06-28 09:33:00,NaN,CHEST,W Chest - Lateral,529,NaN,124.9,3.8,NaN,Rejected,03. Positioning Error
21666,123,NaN,Siemens,Ysio X.Pree,2022-06-28 09:53:00,NaN,SHOULDER,Shoulder - Y-View,452,NaN,72.9,9.5,NaN,Rejected,03. Positioning Error
21667,123,NaN,Siemens,Ysio X.Pree,2022-06-28 10:35:00,NaN,LSPINE,X Lumbar Spine - AP Free Detector,612,NaN,74.8,25.1,NaN,Rejected,03. Positioning Error
21668,123,NaN,Siemens,Ysio X.Pree,2022-06-28 10:34:00,NaN,TSPINE,Thoracic Spine - Lateral Supine,136,NaN,80.9,13.5,NaN,Rejected,06. Anatomy Cutoff


Find unique IDs for the "Body Part" column.

In [31]:
# Extract unique values from the specified column

body_parts = df['Body Part'].unique()
print(body_parts)

['Wrist L' 'Ankle R' 'Wrist R' 'Elbow L' 'Hand L' 'Thumb L' 'Humerus R'
 'Shoulder R' 'Thumb R' 'Hip R' 'Hand R' 'Finger R' 'Tibia/Fibula R'
 'Elbow R' 'Pelvis' 'Forearm L' 'Knee L' 'Hip L' 'Scaphoid R' 'Foot R'
 'Ankle L' 'Calcaneum L' 'Femur R' 'Tibia/Fibula L' 'Shoulder L'
 'Lumbar Spine' 'Humerus L' 'DR Long Leg L' 'Knee R' 'Foot L' 'Scaphoid L'
 'Forearm R' 'Toes L' 'Clavicle L' 'Finger L' 'Femur L'
 'Bilateral Standing Knees' 'Clavicle R' 'Cervical Spine' 'Abdomen'
 'DR Long Leg both' 'Thoracic Spine' 'Sacrum/Coccyx' 'AC Joints' 'Toes R'
 'Bilateral Hands' 'DR Long Leg both Peds' 'Calcaneum R' 'Chest'
 'DR Long Spine Peds' 'Nasal Bones' nan 'DR Long Leg  R' 'DR Long Spine'
 'Unknown RIS Code' 'Emergency Examination' 'CHEST' 'ABDOMEN' 'HUMERUS'
 'ELBOW' 'ARM' 'KNEE' 'PELVIS' 'SPINE' 'LSPINE' 'EXTREMITY' 'ANKLE' 'HIP'
 'SHOULDER' 'TSPINE']


In [32]:
# Predefined categories including "Uncategorized"
BodyPart_Category = {
    'Abdomen': [],
    'Ankle': [],
    'Chest': [],
    'Clavicle': [],
    'C-Spine': [],
    'Elbow': [],
    'Foot': [],
    'Forearm':[],
    'Hand':[],
    'Hip':[],
    'Knee':[],
    'L-Spine':[],
    'Neck':[],
    'Pelvis':[],
    'Shoulder':[],
    'Skull':[],
    'Face':[],
    'Sacrum/Coccyx':[],
    'Tib/Fib':[],
    'T-Spine':[],
    'Wrist':[],
    'Femur':[],
    'Humerus':[],
    'QC/Test':[],
    'Uncategorized': []  # Add a category for skipped entries
}

# Function to categorize body parts
def categorize_body_parts():
    print("Please assign each body part to one of the following categories:")
    print(", ".join(BodyPart_Category.keys()))
    print("\n")

    for body_part in body_parts:
        while True:
            category = input(f"Enter category for '{body_part}' (or press Enter to skip): ")
            if category == "":  # If user skips, add to "Uncategorized"
                BodyPart_Category['Uncategorized'].append(body_part)
                break
            elif category in BodyPart_Category:
                BodyPart_Category[category].append(body_part)
                break
            else:
                print(f"Invalid category! Please choose from the predefined list.")

    print("\nBody Part Categories:")
    print(BodyPart_Category)

# Function to review and categorize uncategorized body parts
def categorize_uncategorized():
    if not BodyPart_Category['Uncategorized']:
        print("\nNo uncategorized body parts to review.")
        return

    print("\nLet's categorize the uncategorized body parts.")
    print("Available categories: ", ", ".join(BodyPart_Category.keys()))

    remaining_uncategorized = []

    for body_part in BodyPart_Category['Uncategorized']:
        while True:
            category = input(f"Enter category for '{body_part}' (or press Enter to leave in 'Uncategorized'): ")
            if category == "":  # If user skips again, leave it in "Uncategorized"
                remaining_uncategorized.append(body_part)
                break
            elif category in BodyPart_Category:
                BodyPart_Category[category].append(body_part)
                break
            else:
                print("Invalid category! Please choose from the predefined list.")

    # Update the uncategorized list
    BodyPart_Category['Uncategorized'] = remaining_uncategorized

    if remaining_uncategorized:
        print("\nThe following body parts are still in 'Uncategorized':")
        print(remaining_uncategorized)
    else:
        print("\nAll body parts have now been categorized.")

# First categorize all body parts
# categorize_body_parts()

# Check and categorize the uncategorized parts
# categorize_uncategorized()


The code below can be used to load an existing dictionary and check if there 

Replace the output file path with the existing dictionary.
Replace the input file path with the new unique values for the Body Part list

In [ ]:
# Load your CSV files (replace 'output.csv' with the actual file path for the output file)
output_file_path = 'output.csv'
output_data = pd.read_csv(output_file_path)

# Extract body parts from the original file (replace 'input.csv' with your actual input file)
input_file_path = 'input.csv'  # Replace with actual input file path
input_data = pd.read_csv(input_file_path)

# Function to load existing categories from output.csv
def load_existing_categories():
    for _, row in output_data.iterrows():
        category = row['Key']
        body_parts = eval(row['Value'])  # Convert string representation of lists back into lists
        if category in BodyPart_Category:
            BodyPart_Category[category] = body_parts

# Function to check and categorize uncategorized body parts from input data
def categorize_new_body_parts():
    all_body_parts = input_data['Body Part'].unique()  # Get all unique body parts from the input data
    categorized_body_parts = [part for parts in BodyPart_Category.values() for part in parts]  # Flatten categorized parts
    
    # Find uncategorized body parts
    uncategorized_body_parts = [part for part in all_body_parts if part not in categorized_body_parts]
    
    # Categorize uncategorized body parts
    if uncategorized_body_parts:
        print("\nThe following body parts have no assigned category:")
        for body_part in uncategorized_body_parts:
            while True:
                category = input(f"Enter category for '{body_part}' (or press Enter to assign to 'Uncategorized'): ")
                if category == "":
                    BodyPart_Category['Uncategorized'].append(body_part)
                    break
                elif category in BodyPart_Category:
                    BodyPart_Category[category].append(body_part)
                    break
                else:
                    print("Invalid category! Please choose from the predefined list.")
    else:
        print("\nNo new uncategorized body parts found.")

# Load existing categories from output.csv
load_existing_categories()

# Check and categorize new body parts
categorize_new_body_parts()

# Function to save updated categories back to output.csv
def save_to_csv():
    updated_data = pd.DataFrame(list(BodyPart_Category.items()), columns=['Key', 'Value'])
    updated_data.to_csv('updated_output.csv', index=False)
    print("\nUpdated categories saved to 'updated_output.csv'.")

# Save the updated categories back to CSV
save_to_csv()

Export the dictionary into a .csv file

In [35]:
data = BodyPart_Category

# Define the CSV file name
file_name = "output.csv"

# Write the dictionary to the CSV file
with open(file_name, mode='w', newline='') as file:
    writer = csv.writer(file)
    # Write header
    writer.writerow(["Key", "Value"])
    # Write data
    for key, value in data.items():
        writer.writerow([key, value])

print(f"Dictionary has been written to {file_name}")

Dictionary has been written to output.csv
